# Konkurs Titanic: Machine Learning from Disaster
Notebook ten poświęcony jest konkursowi ze strony Kaggle dotyczący rozwiązania problemu, kto przetrwał katastrofe Titanica.
Biorąc udział w konkursie otrzymujemy dwa pliki train.csv oraz test.csv, train zawiera dane, kto przetrwał katastrofe i informacje takie jak np. płeć, wiek, ilość rodzeństwa itp.
Plik test, jest plikiem, z którego nasz model ma przewidzieć kto przetrwa katastrofe ucząć się z wykorzystaniem pliku train.csv

Komórka poniżej jest pozostałością z poradnika, który pozwala zacząć zabawe z tym konkursem wykorzystując edytor notebooków na stronie kaggle

In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # algebra liniowa
import pandas as pd # przetwarzanie danych csv, I/O

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [10]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


W powyżej komórce otrzymaliśmy 5 wyników z górnych rzędów tabeli z danych do nauki modelu, można zauważyć, że z tych 5 osób, przeżyły 3 i tymi trzema osobami były kobiety, co może świadczyć, że ważnym czynnikiem, który musi być brany pod uwage jest płeć pasażera.

In [11]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


W powyżej komórce otrzymaliśmy 5 wyników z górnych rzędów tabeli z danych, w których mamy sprawdzić kto przeżyje katastrofe.

In [12]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of women who survived:", rate_women*100)
print("% of men who survived:", rate_men*100)

% of women who survived: 74.20382165605095
% of men who survived: 18.890814558058924


W komórce wyżej zestawione zostały wyniki procentowe z danych do nauki jaki procent pasażerów danej płci przeżył.
Można zauważyc, że przeżyło aż 3/4 kobiet, gdzie mężczyzn prawie 1/5.

In [13]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor


W powyższej komórce importuje narzędzie do wyznaczenia błędu bezwzględnego, oraz narzędzia train_test_split, które rozdziela dane, na dane do nauki sieci oraz dane pozwalające zweryfikować efekty nauki. Dwa ostatnie importy, są to dwa różne modele nauki maszynowej.

# Testowanie modeli:
Następne komórki będą poświęcone testowaniu modeli, tzn. sprawdzania, który z nich posiada mniejszą wartość błędu MAE, oraz przy jakich warunkach ta wartość jest najmniejsza.

Pierwszą rzeczą jaka zostanie zdefiniowana bedzie cel prognozy, w naszym przypadku będzie to ilość osób, które przetrwały, oraz informację, które będą brane pod uwagę podczas nauczania.

In [14]:
y = train_data["Survived"]
features = ["Pclass", "Sex", "SibSp", "Parch"] 
#do nauczania będzie brana pod uwagę klasa, płec, ilość rodzeństwa lub małżonka, ilość rodziców i dzieci na pokładzie. Odrzucone zostały natomiast
#numer biletu, opłata za bilet, numer kabiny (który mógłby mieć duże znaczenie, ponieważ im niżej umieszczona tym były mniejsze szanse dostać się
#na górny pokład, lecz większość pasażerów posiada te rubrykę pustą), ostatnia odrzuconą rzeczą było, w którym porcie dany pasażer dostał sie na 
#pokład Titanica

Kolejnym elementem będzie zdefiniowanie funkcji, która pozwoli nam wyliczyć rózne MAE dla modelu DecisionTreeRegressor oraz RandomForestClassifier

In [15]:
def get_mae_DTR(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

def get_mae_RFC(n_estimators, max_depth, train_X, val_X, train_y, val_y):
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

Przygotowanie danych z pliku train.csv oraz ich rozdzielenie, aby móc sprawdzić jak dwie metody sobie radzą

Przygotowanie danych z pliku train.csv oraz ich rozdzielenie, aby móc sprawdzić jak dwie metody sobie radzą

In [16]:
X = pd.get_dummies(train_data[features])
train_X, val_X, train_y, val_y = train_test_split(X,y)

Przygotowanie dwóch zestawów, jeden do testsowania najlepszej ilości liści dla naszego drzewa, oraz drugi dla modelu losowych lasu, czyli ich ilość drzew oraz ich wielkość.

In [17]:
candidate_max_leaf_nodes = [5, 25, 50, 100, 250, 500]
scores_DTR = {leaf_size: get_mae_DTR(leaf_size, train_X, val_X, train_y, val_y) for leaf_size in candidate_max_leaf_nodes}
best_tree_size = min(scores_DTR, key=scores_DTR.get)

scores_RFC = {}
candidate_max_n_estimators = [5, 25, 50, 100, 250, 500]
candidate_max_dephts = [5, 25, 50, 100, 250, 500]
for n_estimator in candidate_max_n_estimators:
    for depth in candidate_max_dephts:
        scores_RFC[(n_estimator,depth)] = get_mae_RFC(n_estimator, depth, train_X, val_X, train_y, val_y)
(best_n_estimators, best_max_depth) = min(scores_RFC, key=scores_RFC.get)

print("najlepszy rozmiar drzewa:{}\nnajlepsza ilość lasów oraz ich głębia:{}".format(best_tree_size,(best_n_estimators, best_max_depth)))
print("najmniejsze mae dla DTR:{}\nnajmniejsze mae dla RFC:{}".format(scores_DTR[best_tree_size],scores_RFC[(best_n_estimators, best_max_depth)]))

najlepszy rozmiar drzewa:50
najlepsza ilość lasów oraz ich głębia:(50, 5)
najmniejsze mae dla DTR:0.2941094728532896
najmniejsze mae dla RFC:0.2062780269058296


Spoglądając na wyniki, decyduję się wykorzystać RandomForestClassifier z ilością drzew 25 oraz z wielkością 5.

# Prognoza wyniku na podstawie przygotowanych danych testowych
Nadpisuje dane train_X danymi do nauki, a X danymi testowymi, nastepnie definiuje model, na który zdecydowałem się wcześniej oraz wykonuje prognoze wyniku.

In [18]:
train_X = pd.get_dummies(train_data[features])
X = pd.get_dummies(test_data[features])
model = RandomForestClassifier(n_estimators=best_n_estimators, max_depth=best_max_depth, random_state=1)
model.fit(train_X, y)
predictions = model.predict(X)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [19]:
my_data = pd.read_csv("./my_submission.csv")
my_data.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


# Wynik
Wykorzystując ten model osiągnąłem wynik na poziomie 0.77511